In [ ]:
import pandas as pd
import pypsa

n = pypsa.Network(snakemake.input["network"])

df = pd.read_csv(snakemake.input["technologydata"], dtype={"value": float})

# Emissions used as a reference to calculate emissions constraint (from natural gas combined cycle power plant)
# in t_CO2/MWh_e
reference_emissions = df.query(
    "name=='reference emissions' and parameter=='reference emissions'"
).iloc[0]["value"]

# total electricity demand in MWh_e
total_demand = n.loads_t["p_set"]["load"].sum()

# Calculate emissions in t_CO2 if total demand is served from NG CC PPs (max emissions)
total_emissions = reference_emissions * total_demand

# Calculate emission limit (constraint) based on wildcard value (p.u. of max emissions)
emission_limit = total_emissions * float(snakemake.wildcards["co2limit"])

# Add constraint to network and save
n.add(
    "GlobalConstraint",
    "CO2Limit",
    carrier_attribute="co2_emissions",
    sense="<=",
    constant=emission_limit,
)

n.export_to_netcdf(snakemake.output["network"])